# Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import optuna


from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import f1_score

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Data Loading

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
train

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience,blueWins
0,0,0,5,8,6,0,0,14536,17256,0
1,1,1,10,1,5,0,0,14536,17863,0
2,2,0,3,10,2,0,0,17409,17256,0
3,3,1,7,10,8,0,0,19558,18201,0
4,4,0,4,9,4,0,0,17409,17256,0
...,...,...,...,...,...,...,...,...,...,...
7995,9993,1,9,6,14,0,0,18513,18201,1
7996,9994,0,3,10,4,0,0,17381,19797,0
7997,9995,1,9,3,9,2,1,18274,18491,1
7998,9997,0,5,10,6,1,1,18274,18491,1


In [4]:
test

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience
0,9,0,7,6,6,0,0,16961,18201
1,15,0,6,6,6,2,1,18513,18021
2,18,1,6,4,3,0,0,13475,17256
3,23,0,5,4,7,0,0,17409,17256
4,31,0,10,8,9,0,0,18117,18472
...,...,...,...,...,...,...,...,...,...
1995,9971,0,6,3,7,0,0,17409,17256
1996,9980,0,4,4,4,0,0,17409,17256
1997,9983,1,6,3,3,1,1,18513,18201
1998,9996,1,10,9,9,1,1,18513,18201


# Feature

- 各特長量の説明

| ヘッダ名称            | データ型 | 説明                                                       |
|---------------------|-------|----------------------------------------------------------|
| gameId             | int   | ゲームID                                                    |
| blueFirstBlood      | int   | ゲームの最初のキル。青チームが最初のキルを行った場合は1、それ以外の場合は0 |
| blueKills           | int   | 青チームによって殺された敵の数                                     |
| blueDeaths          | int   | 青チームの死亡者数                                            |
| blueAssists         | int   | 青チームのキルアシストの数                                      |
| blueEliteMonsters   | int   | 青チームによって殺されたエリートモンスターの数（ドラゴンとヘラルド）       |
| blueDragons         | int   | 青チームによって殺されたドラゴンの数                               |
| blueTotalGold       | int   | 青チームの得たゴールド合計                                       |
| blueTotalExperience | int   | 青チームの得た経験値合計                                       |
| blueWins            | int   | 目的変数（青チームが勝った場合は1、それ以外の場合は0。）                |


gameIdとgoldは相関無さそう

# Feature Engineering

- Add new feature

In [5]:
def add_features(df):
    df['assistRate'] = df['blueAssists'] / (df['blueKills'] + 1)
    df['blueHerald'] = (df['blueEliteMonsters'] - df['blueDragons']).clip(lower=0)
    df['eliteMonsterDragonRatio'] = df['blueEliteMonsters'] / (df['blueDragons'] + 1)
    df['logTotalGold'] = np.log1p(df['blueTotalGold'])
    df['logTotalExperience'] = np.log1p(df['blueTotalExperience'])
    df['firstBloodKillRatio'] = df['blueFirstBlood'] * df['blueKills']
    df['totalCombatPoints'] = df['blueKills'] + df['blueDeaths'] + df['blueAssists']

    total_combat_actions = df['blueKills'] + df['blueDeaths'] + df['blueAssists']
    df['killRatio'] = df['blueKills'] / total_combat_actions
    df['deathRatio'] = df['blueDeaths'] / total_combat_actions
    df['assistRatio'] = df['blueAssists'] / total_combat_actions

    df['teamStrength'] = df['blueKills'] + df['blueAssists'] + df['blueEliteMonsters']
    df['dragonKillImpact'] = df['blueDragons'] / (df['blueKills'] + 1)
    df['eliteMonsterUtilization'] = df['blueEliteMonsters'] / (df['blueKills'] + df['blueAssists'] + 1)
    df['goldExperienceRatio'] = df['blueTotalGold'] / (df['blueTotalExperience'] + 1)
    df['teamEfficiency'] = (df['blueKills'] + df['blueAssists'] + df['blueEliteMonsters']) / (df['blueDeaths'] + 1)
    df['killToMonsterRatio'] = df['blueKills'] / (df['blueEliteMonsters'] + 1)
    df['avgGoldPerKill'] = df['blueTotalGold'] / (df['blueKills'] + 1)
    df['expToKillRatio'] = df['blueTotalExperience'] / (df['blueKills'] + 1)
    df['killsMinusDeaths'] = df['blueKills'] - df['blueDeaths']
    df['killDeathRatio'] = df['blueKills'] / (df['blueKills'] + df['blueDeaths'])
    df['avgExperiencePerKill'] = df['blueTotalExperience'] / (df['blueKills'] + 1)
    df['killDeathDiff'] = (df['blueKills'] - df['blueDeaths']).clip(lower=0)
    df['eliteMonsterHeraldRatio'] = df['blueEliteMonsters'] / (df['blueHerald'] + 1)
    df['goldXexperience'] = np.log1p(df['blueTotalGold'] * df['blueTotalExperience'])
    df['goldPerKill'] = df['blueTotalGold'] / (df['blueKills'] + 1)  # ゼロ除算を防ぐために+1
    df['experiencePerKill'] = df['blueTotalExperience'] / (df['blueKills'] + 1)  # ゼロ除算を防ぐために+1
    df['eliteMonstersPerGold'] = df['blueEliteMonsters'] / df['blueTotalGold']
    df['eliteMonstersPerExperience'] = df['blueEliteMonsters'] / df['blueTotalExperience']

    
    return df

- Apply data

In [6]:
train = add_features(train)
test = add_features(test)

# model

- LightGBM

In [7]:


# データの準備
X = train.drop(['blueWins', 'gameId'], axis=1)
y = train['blueWins']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

test_drop_gameId = test.drop('gameId', axis=1)


# KFoldの設定
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

# 各フォールドでのスコアを保存するリスト
fold_scores = []

# testを保存するリスト
test_predictions = []

for train_index, val_index in kf.split(X):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    # Optunaでのハイパーパラメータチューニング
    def objective(trial):
        param = {
            'objective': 'binary',
            'metric': 'binary_error',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
            'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
        }

        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

        verbose_eval = 0
        verbose = lgb.log_evaluation(verbose_eval)

        # 早期停止の設定
        early_stopping = lgb.early_stopping(stopping_rounds=10, verbose=True)
        model = lgb.train(param, lgb_train, num_boost_round=1000, valid_sets=[lgb_val], callbacks=[early_stopping, verbose])
        y_pred = model.predict(X_val, num_iteration=model.best_iteration)
        y_pred = (y_pred > 0.5).astype(int)
        accuracy = accuracy_score(y_val, y_pred)

        return accuracy

    # Optunaでのハイパーパラメータチューニング
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    # 各フォールドの最適パラメータでモデルを再学習
    best_params = study.best_params
    best_params['objective'] = 'binary'
    best_params['metric'] = 'binary_error'
    best_params['verbosity'] = -1

    lgb_train_fold = lgb.Dataset(X_train_fold, y_train_fold)
    model_fold = lgb.train(best_params, lgb_train_fold, num_boost_round=study.best_trial.number)

    # 各フォールドのモデル性能を評価
    y_val_pred = model_fold.predict(X_val_fold)
    y_val_pred_int = (y_val_pred > 0.5).astype(int)
    fold_accuracy = accuracy_score(y_val_fold, y_val_pred_int)
    fold_scores.append(fold_accuracy)

    # テストデータセットでの予測
    y_test_pred_fold = model_fold.predict(test_drop_gameId)
    test_predictions.append(y_test_pred_fold)

# 全フォールドの平均スコアを計算
average_score = sum(fold_scores) / n_splits
print(f"Average accuracy across all folds: {average_score}")


[I 2024-01-25 09:33:04,048] A new study created in memory with name: no-name-ab8d2c68-1c2d-4788-a205-20761eae1d8d
[I 2024-01-25 09:33:04,115] Trial 0 finished with value: 0.79875 and parameters: {'lambda_l1': 0.4009290218772377, 'lambda_l2': 0.0004396615017244955, 'num_leaves': 60, 'feature_fraction': 0.6201472238263483, 'bagging_fraction': 0.8924902874434768, 'bagging_freq': 6, 'min_child_samples': 74}. Best is trial 0 with value: 0.79875.
[I 2024-01-25 09:33:04,167] Trial 1 finished with value: 0.789375 and parameters: {'lambda_l1': 1.0763856700487895, 'lambda_l2': 2.5283158844887416e-07, 'num_leaves': 80, 'feature_fraction': 0.5779005214777078, 'bagging_fraction': 0.43741639476747396, 'bagging_freq': 2, 'min_child_samples': 65}. Best is trial 0 with value: 0.79875.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	valid_0's binary_error: 0.20125
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	valid_0's binary_error: 0.210625
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[40]	valid_0's binary_error: 0.195625


[I 2024-01-25 09:33:04,338] Trial 2 finished with value: 0.804375 and parameters: {'lambda_l1': 2.3444748774623005e-05, 'lambda_l2': 0.00013041495968429525, 'num_leaves': 62, 'feature_fraction': 0.7717750010789071, 'bagging_fraction': 0.8262393175356348, 'bagging_freq': 5, 'min_child_samples': 95}. Best is trial 2 with value: 0.804375.
[I 2024-01-25 09:33:04,448] Trial 3 finished with value: 0.801875 and parameters: {'lambda_l1': 0.0003853142886558033, 'lambda_l2': 1.636103107498399e-05, 'num_leaves': 121, 'feature_fraction': 0.5713210051898221, 'bagging_fraction': 0.9481695765366193, 'bagging_freq': 2, 'min_child_samples': 95}. Best is trial 2 with value: 0.804375.
[I 2024-01-25 09:33:04,503] Trial 4 finished with value: 0.80125 and parameters: {'lambda_l1': 0.6644982038799359, 'lambda_l2': 0.019190597433624817, 'num_leaves': 85, 'feature_fraction': 0.7027154116059522, 'bagging_fraction': 0.6109266932521388, 'bagging_freq': 4, 'min_child_samples': 38}. Best is trial 2 with value: 0.80

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[20]	valid_0's binary_error: 0.198125
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[15]	valid_0's binary_error: 0.19875
Training until validation scores don't improve for 10 rounds


[I 2024-01-25 09:33:04,588] Trial 5 finished with value: 0.803125 and parameters: {'lambda_l1': 0.009625831017939979, 'lambda_l2': 1.0581785785525955e-08, 'num_leaves': 118, 'feature_fraction': 0.6297146598182902, 'bagging_fraction': 0.5617546055719753, 'bagging_freq': 4, 'min_child_samples': 68}. Best is trial 2 with value: 0.804375.
[I 2024-01-25 09:33:04,628] Trial 6 finished with value: 0.78 and parameters: {'lambda_l1': 3.2613648908001025, 'lambda_l2': 0.013067091139271668, 'num_leaves': 152, 'feature_fraction': 0.4706588125464564, 'bagging_fraction': 0.587815494751776, 'bagging_freq': 1, 'min_child_samples': 84}. Best is trial 2 with value: 0.804375.
[I 2024-01-25 09:33:04,694] Trial 7 finished with value: 0.7925 and parameters: {'lambda_l1': 6.666038275885567e-06, 'lambda_l2': 0.06599556012214772, 'num_leaves': 159, 'feature_fraction': 0.49837082082712697, 'bagging_fraction': 0.5391331658726705, 'bagging_freq': 3, 'min_child_samples': 92}. Best is trial 2 with value: 0.804375.


...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[10]	valid_0's binary_error: 0.196875
Training until validation scores don't improve for 10 rounds


[I 2024-01-25 09:34:31,249] Trial 97 finished with value: 0.805 and parameters: {'lambda_l1': 0.00020924573907534088, 'lambda_l2': 2.9164637644852364e-08, 'num_leaves': 217, 'feature_fraction': 0.41287471077611204, 'bagging_fraction': 0.9339997662612614, 'bagging_freq': 2, 'min_child_samples': 24}. Best is trial 11 with value: 0.8125.


Early stopping, best iteration is:
[19]	valid_0's binary_error: 0.195
Training until validation scores don't improve for 10 rounds


[I 2024-01-25 09:34:31,442] Trial 98 finished with value: 0.811875 and parameters: {'lambda_l1': 8.40663045981894e-07, 'lambda_l2': 1.375847806653118e-07, 'num_leaves': 246, 'feature_fraction': 0.43370271799382554, 'bagging_fraction': 0.9177989561223486, 'bagging_freq': 3, 'min_child_samples': 39}. Best is trial 11 with value: 0.8125.
[I 2024-01-25 09:34:31,583] Trial 99 finished with value: 0.798125 and parameters: {'lambda_l1': 7.216625641066607e-08, 'lambda_l2': 1.7325685957573522e-07, 'num_leaves': 80, 'feature_fraction': 0.5859114174053406, 'bagging_fraction': 0.9194759385648581, 'bagging_freq': 3, 'min_child_samples': 94}. Best is trial 11 with value: 0.8125.


Early stopping, best iteration is:
[21]	valid_0's binary_error: 0.188125
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[11]	valid_0's binary_error: 0.201875
Average accuracy across all folds: 0.78275


- KFoldの結果確認

In [8]:
# 全フォールドの平均スコアを計算
average_score = sum(fold_scores) / n_splits
print(f"Average accuracy across all folds: {average_score}")

Average accuracy across all folds: 0.78275


- testの平均を計算

In [9]:
# 予測の平均を計算
y_test_pred_avg = np.mean(test_predictions, axis=0)

# 最終的な予測結果を整数に変換
y_test_pred_int = (y_test_pred_avg > 0.5).astype(int)

- testにy_test_pred結合

In [10]:
output = f'../data/output/KFold_val{average_score:.4f}.csv'

In [11]:
submit = pd.DataFrame({
    '0': test['gameId'],
    '1': y_test_pred_int
})

# カラムヘッダー消去して出力
# ヘッダーなしでCSVファイルとして保存
submit.to_csv(output, header=False, index=False)

- 出力csvチェック

In [12]:
check = pd.read_csv(output)

In [13]:
check

,9,1
0,15,1
1,18,0
2,23,0
3,31,0
4,32,1
...,...,...
1994,9971,0
1995,9980,0
1996,9983,1
1997,9996,1
